In [ ]:
from IPython.display import display, clear_output
import ipywidgets as widgets

from scripts.loader import extract_text_from_uploaded_file
from scripts.prompt_runner import run_prompt_with_memory, reset_history
from scripts.response_parser import parse_response
from scripts.utils import read_config

config = read_config("config/config.yaml")

upload_widget = widgets.FileUpload(accept='.pdf,.html', multiple=True, description='Upload Reports')
file_count_label = widgets.Label(value='No files uploaded yet.')
prompt_box = widgets.Textarea(value='Summarize the following research report: [RESEARCH_REPORT]', layout=widgets.Layout(width='100%', height='150px'))
follow_up_toggle = widgets.Checkbox(value=False, description='Is Follow-Up?')
run_button = widgets.Button(description='Run Prompt')
reset_button = widgets.Button(description='Reset Conversation')
output_area = widgets.Output()

display(upload_widget, file_count_label, prompt_box, follow_up_toggle, run_button, reset_button, output_area)

def on_upload_change(change):
    num_files = len(upload_widget.value)
    file_count_label.value = f'{num_files} file(s) uploaded.' if num_files else 'No files uploaded yet.'
upload_widget.observe(on_upload_change, names='value')

def on_run_clicked(b):
    with output_area:
        clear_output()
        uploaded_texts = []
        if not follow_up_toggle.value:
            for fileinfo in upload_widget.value.values():
                uploaded_texts.append(extract_text_from_uploaded_file(fileinfo))
        prompt = prompt_box.value
        response = run_prompt_with_memory(prompt, uploaded_texts, config, is_followup=follow_up_toggle.value)
        print("LLM Response:\n", response)
        if config['defaults']['parse_response'] and not follow_up_toggle.value:
            print("\nParsed JSON:")
            print(parse_response(response))

def on_reset_clicked(b):
    with output_area:
        clear_output()
        reset_history()
        print("Conversation history cleared.")

run_button.on_click(on_run_clicked)
reset_button.on_click(on_reset_clicked)